In [5]:
import pandas as pd

#Cargar el CSV
file_path = "data/new_data.csv"
df = pd.read_csv(file_path, dtype={'Postal code': str})

print(df.columns)

Index(['Date', 'Geo Location', 'Name test', 'Value test', 'Pet sex', 'Pet age',
       'Country', 'City', 'Postal code'],
      dtype='object')


In [6]:



#Normalizar el Formato de las Columnas
df.columns = df.columns.str.strip().str.replace(" ", "_").str.lower()  # Eliminar espacios y estandarizar nombres

df["date"] = df["date"].astype(str).str.replace(r'[^\w\s]', '', regex=True)  # Eliminar caracteres especiales
df["date"] = pd.to_datetime(df["date"], errors='coerce').dt.strftime('%Y-%m-%d %H:%M:%S')

#Eliminar Filas Sin Código Postal ni Geolocalización
df_cleaned = df.dropna(subset=['postal_code'])

val_no_deseados = ['Dudoso', 'Error', 'Invalido', 'Inválido']
df_cleaned = df[~df['value_test'].isin(val_no_deseados)]

#Definir Valores Positivos
value_test_mapping = {
    'Dudoso': -2,
    'Error': -1,
    'Invalido': -1,
    'Inválido': -1,
    'Negativo': 0,
    'Positivo': 1,
    'Positivo Fuerte': 1
}

# Aplicar mapeo a la columna value_test
df_cleaned['test_result'] = df_cleaned['value_test'].map(value_test_mapping)
df_cleaned['postal_code'] = df_cleaned['postal_code']

df_cleaned.loc[df_cleaned['postal_code'] == '6430', 'City'] = 'Zalamea De La Serena'
#Agrupar por Código Postal
df_grouped = df_cleaned.groupby("postal_code").agg(
    total_tests=('test_result', 'count'),
    total_positives=('test_result', lambda x: (x == 1).sum())
).reset_index()

#Guardar los Resultados
df_cleaned.to_csv("data/preprocessed_data.csv", index=False)  # CSV limpio
df_grouped.to_csv("data/postal_codes_results.csv", index=False)  # Agrupado por código postal


C:\Users\pablo\AppData\Local\Temp\ipykernel_9964\808124312.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['test_result'] = df_cleaned['value_test'].map(value_test_mapping)
C:\Users\pablo\AppData\Local\Temp\ipykernel_9964\808124312.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['postal_code'] = df_cleaned['postal_code']
C:\Users\pablo\AppData\Local\Temp\ipykernel_9964\808124312.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.